In [ ]:
# ========================= Q1: Train a Simple Neural Network & Perform EDA =========================
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Print dataset shape
print(f"Training Data Shape: {x_train.shape}, Labels: {y_train.shape}")
print(f"Test Data Shape: {x_test.shape}, Labels: {y_test.shape}")

# CIFAR-10 class labels
class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer',
               'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

# Display 9 sample images with labels
plt.figure(figsize=(8, 8))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(x_train[i])
    plt.title(f"{class_names[y_train[i][0]]}")
    plt.axis('off')
plt.show()

# Plot label distribution
unique, counts = np.unique(y_train, return_counts=True)
plt.figure(figsize=(8, 5))
sns.barplot(x=unique, y=counts, palette="viridis")
plt.xlabel("Class Label")
plt.ylabel("Count")
plt.title("Distribution of Labels in Training Data")
plt.show()

# Normalize pixel values (scale between 0 and 1)
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flatten images from 32x32x3 to 3072 (input layer)
x_train = x_train.reshape(-1, 3072)
x_test = x_test.reshape(-1, 3072)

# Define a simple ANN model
def create_basic_model():
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(3072,)),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train basic ANN model
model_basic = create_basic_model()
history_basic = model_basic.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

# Evaluate basic model
basic_loss, basic_acc = model_basic.evaluate(x_test, y_test, verbose=2)
print(f"Test Accuracy (Basic ANN): {basic_acc:.4f}")

# ========================= Q2(a): Train Model with and without Dropout & Compare Accuracy =========================
def create_dropout_model():
    model = keras.Sequential([
        keras.layers.Dense(256, activation='relu', input_shape=(3072,)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train model WITHOUT dropout
model_no_dropout = create_dropout_model()
history_no_dropout = model_no_dropout.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test))

# Train model WITH dropout
model_with_dropout = create_dropout_model()
history_with_dropout = model_with_dropout.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test))

# Evaluate models
loss_no_dropout, acc_no_dropout = model_no_dropout.evaluate(x_test, y_test, verbose=2)
loss_with_dropout, acc_with_dropout = model_with_dropout.evaluate(x_test, y_test, verbose=2)

print(f"Test Accuracy WITHOUT Dropout: {acc_no_dropout:.4f}")
print(f"Test Accuracy WITH Dropout (0.3): {acc_with_dropout:.4f}")

# Plot accuracy & loss comparison
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_no_dropout.history['val_accuracy'], label='No Dropout')
plt.plot(history_with_dropout.history['val_accuracy'], label='With Dropout')
plt.xlabel('Epochs')
plt.ylabel('Test Accuracy')
plt.legend()
plt.title('Test Accuracy Comparison')

plt.subplot(1, 2, 2)
plt.plot(history_no_dropout.history['val_loss'], label='No Dropout')
plt.plot(history_with_dropout.history['val_loss'], label='With Dropout')
plt.xlabel('Epochs')
plt.ylabel('Test Loss')
plt.legend()
plt.title('Test Loss Comparison')

plt.show()

# ========================= Q2(b): Evaluate Model & Save/Reload =========================
# Make predictions on test data
y_pred = model_with_dropout.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Compute classification metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

# Print the metrics
print("="*40)
print("|   Metric      |   Value    |")
print("="*40)
print(f"| Accuracy      |   {accuracy:.4f}   |")
print(f"| Precision     |   {precision:.4f}   |")
print(f"| Recall        |   {recall:.4f}   |")
print(f"| F1 Score      |   {f1:.4f}   |")
print("="*40)

# Save model
model_path = "cifar10_ann_model.h5"
model_with_dropout.save(model_path)
print(f"Model saved at {model_path}")

# Reload model
loaded_model = keras.models.load_model(model_path)
print("Model reloaded successfully!")

# Evaluate reloaded model
reloaded_loss, reloaded_acc = loaded_model.evaluate(x_test, y_test, verbose=2)
print(f"Test Accuracy (Reloaded Model): {reloaded_acc:.4f}")

In [ ]:
#Q1: Training a Simple Neural Network on CIFAR-10
#1. Simple Neural Network for CIFAR-10
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np

# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Simple CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=10,
                    validation_data=(x_test, y_test))

# Evaluate on test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc:.4f}')
#Interpretation:
#The simple neural network achieves about 67% accuracy on the CIFAR-10 test set. This is a decent baseline performance for a simple model. The training accuracy (76.15%) is higher than the test accuracy, indicating some overfitting to the training data.

#2a) Including Dropout Layer and Comparing Accuracy
# Model with dropout
model_dropout = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model_dropout.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

history_dropout = model_dropout.fit(x_train, y_train, epochs=10,
                                    validation_data=(x_test, y_test))

# Evaluate dropout model
test_loss_dropout, test_acc_dropout = model_dropout.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy with dropout: {test_acc_dropout:.4f}')

# Compare with original model
print(f'Original test accuracy: {test_acc:.4f}')
print(f'Improvement: {test_acc_dropout - test_acc:.4f}')

# Interpretation:
# The model with dropout layers achieved 69.86% accuracy compared to 66.98% without dropout, showing an improvement of about 2.88%. The dropout layers helped reduce overfitting as evidenced by:

# The smaller gap between training (69.28%) and test accuracy (69.86%)

# The improved generalization performance on the test set

#2b) Evaluation Metrics, Saving and Reloading Model
from sklearn.metrics import classification_report
import numpy as np

# Train model (same as in part 1)
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Get predictions for evaluation metrics
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_true, y_pred_classes, target_names=[
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]))

# Save the model
model.save('cifar10_model.h5')

# Reload the model and verify
loaded_model = tf.keras.models.load_model('cifar10_model.h5')
_, loaded_acc = loaded_model.evaluate(x_test, y_test, verbose=0)
print(f'\nOriginal model accuracy: {test_acc:.4f}')
print(f'Reloaded model accuracy: {loaded_acc:.4f}')


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - accuracy: 0.7494 - loss: 0.7269 - val_accuracy: 0.6995 - val_loss: 0.8841
Epoch 7/10


In [ ]:
# 2b) Plotting Loss and Accuracy
def plot_history(history, history_dropout):
    # Accuracy plot for both models
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train accuracy (no dropout)')
    plt.plot(history.history['val_accuracy'], label='Test accuracy (no dropout)')
    plt.plot(history_dropout.history['accuracy'], label='Train accuracy (with dropout)')
    plt.plot(history_dropout.history['val_accuracy'], label='Test accuracy (with dropout)')
    plt.title('Model Accuracy Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')

    # Loss plot for both models
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train loss (no dropout)')
    plt.plot(history.history['val_loss'], label='Test loss (no dropout)')
    plt.plot(history_dropout.history['loss'], label='Train loss (with dropout)')
    plt.plot(history_dropout.history['val_loss'], label='Test loss (with dropout)')
    plt.title('Model Loss Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

# Plotting the comparison of accuracy and loss
plot_history(history, history_dropout)